### 0. 환경 설정

In [1]:
#Installing py3Dmol using pip
!pip install py3Dmol
#Installing biopython using pip
!pip install biopython
#Installing pdb2pqr v3.0 using pip
!pip install pdb2pqr
#We will also install kora for using RDkit
!pip install kora

!pip install rdkit

In [2]:
#Importing py3Dmol for safety
import py3Dmol

In [3]:
#Checking that pdb2pqr was properly installed
!pdb2pqr30 -h

usage: pdb2pqr [-h] [--ff {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
               [--userff USERFF] [--clean] [--nodebump] [--noopt]
               [--keep-chain] [--assign-only]
               [--ffout {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
               [--usernames USERNAMES] [--apbs-input APBS_INPUT]
               [--pdb-output PDB_OUTPUT] [--ligand LIGAND] [--whitespace]
               [--neutraln] [--neutralc] [--drop-water] [--include-header]
               [--titration-state-method {propka}] [--with-ph PH]
               [-f FILENAMES] [-r REFERENCE] [-c CHAINS] [-i TITRATE_ONLY]
               [-t THERMOPHILES] [-a ALIGNMENT] [-m MUTATIONS] [-p PARAMETERS]
               [--log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}] [-o PH]
               [-w WINDOW WINDOW WINDOW] [-g GRID GRID GRID]
               [--mutator MUTATOR] [--mutator-option MUTATOR_OPTIONS] [-d]
               [-l] [-k] [-q] [--protonate-all] [--version]
               input_path output_pqr

PDB2PQR v3.

### 1. Receptor : LRRK2 G2019S mutant 준비

In [4]:
#Downloading the PDB files using biopython
import os
from Bio.PDB import *
pdbid = ['7li3']  #LRRK2의 PDB id는 7li3
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [7]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
singlepath = Path("/content/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(singlepath):
  print("path already exists")
if not os.path.exists(singlepath):
  os.mkdir(singlepath)
  print("path was succesfully created")

#Now we assign a variable "protein" with the name and extension of our pdb
protein = "7li3.pdb"

#And we use the following script to selectively print the lines that contain the
#string "ATOM" and "TER" into a new file inside our recently created folder
with open( "7li3_prot.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")


path already exists
file successfully created


### 2. Binding Box 찾기

In [9]:
import numpy as np
with open('7li3.pdb', 'r') as f:
  ligand_geom = []
  for l in f:
    if l.startswith('HETATM') and l[17:20]=='ATP':   #4K4(LRRK2-IN-1)이 결합해있음
      x, y, z = float(l[30:38]), float(l[38:46]), float(l[46:54])
      print(x,y,z)
      ligand_geom.append([x,y,z])
  print("Ligand Geom:\n", ligand_geom)
  ligand_geom = np.array(ligand_geom)
  ligand_center = ligand_geom.mean(axis=0)
  print(f"Geometric Center of a ligand: {ligand_center[0]:8.3f} {ligand_center[1]:8.3f} {ligand_center[2]:8.3f}")

224.039 210.729 243.528
223.509 209.362 243.172
223.134 211.532 244.431
225.497 210.747 243.918
222.604 211.778 241.377
221.881 212.898 242.084
221.918 210.453 241.159
223.999 211.533 242.137
223.716 211.331 238.863
222.595 210.573 238.198
224.861 210.607 239.523
223.108 212.336 239.958
224.319 212.39 237.822
225.625 212.903 238.041
225.878 214.017 237.039
226.237 213.459 235.781
224.657 214.887 236.817
224.998 216.229 237.158
224.362 214.827 235.337
224.278 216.152 234.812
225.546 214.11 234.723
225.081 213.09 233.762
224.377 211.987 234.06
224.115 211.264 232.944
224.662 211.919 231.907
224.757 211.711 230.448
224.197 210.627 229.865
225.421 212.638 229.726
225.98 213.717 230.302
225.93 213.969 231.618
225.295 213.121 232.453
Ligand Geom:
 [[224.039, 210.729, 243.528], [223.509, 209.362, 243.172], [223.134, 211.532, 244.431], [225.497, 210.747, 243.918], [222.604, 211.778, 241.377], [221.881, 212.898, 242.084], [221.918, 210.453, 241.159], [223.999, 211.533, 242.137], [223.716, 211.3